In [1]:
from os import listdir
from os.path import isfile, join
# from Utils import Load, Write
import pandas as pd
import numpy as np
from scipy import stats
# import math
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
# from matplotlib import cm
# import plotly.plotly as py
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
from pop_calcs import *
from pop_organize import *
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

In [2]:
#########################
#    main function      #
#########################

##generate list of sample file names:
onlyfiles = [f for f in listdir("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles") if isfile(join("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles", f))]
onlyfiles = [datafile for datafile in onlyfiles if datafile.startswith ('HIP') and datafile.endswith('.csv')]
sample_names=[re.sub('.csv', '', datafile) for datafile in onlyfiles] ## generate list of 
                                                                      ## sample names out 
                                                                      ##of the sample file names
df_file_names,samples_with_df=get_sample_with_dfs() ## check which samples have df
generate_dfs=False ##False=load them from pickles, True=generate them. 

df_n=2 ## define number of dfs (prod, non-prod) *** check for update!***

getting list of samples that have dfs...
finished getting list of samples that have dfs


In [1]:
##length functions: 

'vGeneName','vFamilyName', 'jGeneName', 'jFamilyName','dGeneName', 'v-j','v-j-family']

def mean_cdr3(df):
    mean_cdr3=np.mean(df['cdr3Length'])
    return mean_cdr3

def sv_cdr3(df):
    sv_cdr3=np.std(df['cdr3Length'])/np.mean(df['cdr3Length'])
    return sv_cdr3

def mean_vDeletion(df):
    mean_vDeletion=np.mean(df['vDeletion'])
    return mean_vDeletion

def sv_vDeletion(df):
    sv_vDeletion=np.std(df['vDeletion'])/np.mean(df['vDeletion'])
    return sv_vDeletion

def mean_n1Insertion(df):
    mean_n1Insertion=np.mean(df['n1Insertion'])
    return mean_n1Insertion

def sv_n1Insertion(df):
    sv_n1Insertion=np.std(df['n1Insertion'])/np.mean(df['n1Insertion'])
    return sv_n1Insertion

def mean_d5Deletion(df):
    mean_d5Deletion=np.mean(df['d5Deletion'])
    return mean_d5Deletion

def sv_d5Deletion(df):
    sv_d5Deletion=np.std(df['d5Deletion'])/np.mean(df['d5Deletion'])
    return sv_d5Deletion

def mean_d3Deletion(df):
    mean_d3Deletion=np.mean(df['d3Deletion'])
    return mean_d3Deletion

def sv_d3Deletion(df):
    sv_d3Deletion=np.std(df['d3Deletion'])/np.mean(df['d3Deletion'])
    return sv_d3Deletion

def mean_n2Insertion(df):
    mean_n2Insertion=np.mean(df['n2Insertion'])
    return mean_n2Insertion

def sv_n2Insertion(df):
    sv_n2Insertion=np.std(df['n2Insertion'])/np.mean(df['n2Insertion'])
    return sv_n2Insertion

def mean_jDeletion(df):
    mean_jDeletion=np.mean(df['jDeletion'])
    return mean_jDeletion

def sv_jDeletion(df):
    sv_jDeletion=np.std(df['jDeletion'])/np.mean(df['jDeletion'])
    return sv_jDeletion


SyntaxError: invalid syntax (<ipython-input-1-00fe2da0ed50>, line 3)

In [5]:
length_func_list=[mean_cdr3, sv_cdr3, mean_vDeletion, sv_vDeletion, mean_n1Insertion, sv_n1Insertion, mean_d5Deletion, sv_d5Deletion, 
                  mean_d3Deletion, sv_d3Deletion, mean_n2Insertion, sv_n2Insertion, mean_jDeletion, sv_jDeletion]

In [6]:
def generate_res_DF(sample_names, function_list, df_n):
    print 'generating result dataframe...'
    result_df=pd.DataFrame({'Sample': sample_names})
    result_df=result_df.set_index('Sample')
    function_names=[f.__name__ for f in function_list]
    for function in function_names:
        for i in range(df_n):
            col_title=function+'_df_%s' %str(i)
            result_df[col_title]=np.NaN
    print 'finished generating result dataframe'
    return result_df


In [7]:
length_res_df=generate_res_DF(samples_with_df, length_func_list, df_n)

generating result dataframe...
finished generating result dataframe


In [8]:
def get_sample_data(sample_name, generate_dfs): 
## this function generates dfs (general, only productive and only non productive) for each sample, and save
## as pickles. alternatively, it loads these dfs from pickles. 
    
    print 'getting sample data...'
    
    if generate_dfs:
        sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" %sample_name)  
        sample_df_prod = sample_df[sample_df['sequenceStatus'] == 'In']
        sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']
        
        pickle.dump(sample_df, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_%s' %sample_name, "wb"))
        pickle.dump(sample_df_prod, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_prod_%s' %sample_name, "wb"))
        pickle.dump(sample_df_non_prod, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_non_prod_%s' %sample_name, "wb"))
    else:
        sample_df= pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_%s' %sample_name,"rb" ))
        sample_df_prod= pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_prod_%s' %sample_name,"rb" ))
        sample_df_non_prod= pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_non_prod_%s' %sample_name,"rb" ))
    
    print 'finished getting sample data'
    return sample_df, sample_df_prod, sample_df_non_prod

#_________________________________________________________________

In [9]:
def calculate_res_df(result_df,function_list, sample_name, sample_df_prod, sample_df_non_prod, **kwargs):
    ind=0
    df_list=[sample_df_prod, sample_df_non_prod] ## df list is generated here and doesnt come 
                                                 ## as input as it contains the specfic dfs
                                                 ##for each sample   
    
    for j, func in enumerate(function_list):
        for i, df in enumerate(df_list):
            result=func(df, **kwargs)
            result_df.loc[sample_name, result_df.columns[ind]]=result
            ind+=1
    return result_df



In [10]:
n_samples=4

In [ ]:
n=1
for d in range(len(samples_with_df[:n_samples])): ##***change here for more samples!***
    sample_name=samples_with_df[d]
    print n
    print sample_name
## extract prod and non-prod dfs: 
    sample_df, sample_df_prod, sample_df_non_prod=get_sample_data(sample_name, generate_dfs)
    percent_prod=perc_prod(sample_df)
#    perProd_res_df.loc[sample_name, 'perc_prod_df_0']=percent_prod
#    general_res_df=calculate_res_df(general_res_df,general_function_list, sample_name, sample_df_prod, sample_df_non_prod) 
#    clonality_res_df=calculate_res_df(clonality_res_df,clonality_func_list, sample_name, sample_df_prod, sample_df_non_prod)
#    diversity_res_df=calculate_res_df(diversity_res_df,diversity_func_list, sample_name, sample_df_prod, sample_df_non_prod)
    length_res_df=calculate_res_df(length_res_df,length_func_list, sample_name, sample_df_prod, sample_df_non_prod)
    n=n+1
#with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/general_res_df_%s_samples' %n_samples, "wb") as f1:
#    pickle.dump(general_res_df, f1)
#with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/perProd_res_df_%s_samples' %n_samples, "wb") as f2:
#    pickle.dump(perProd_res_df, f2)
#with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/clonality_res_df_%s_samples' %n_samples, "wb") as f3:
#    pickle.dump(clonality_res_df, f3)
#with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/diversity_res_df_%s_samples' %n_samples, "wb") as f4:
    pickle.dump(diversity_res_df, f4)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/length_res_df_%s_samples' %n_samples, "wb") as f5:
    pickle.dump(length_res_df, f5)


1
HIP11513
getting sample data...
